In [89]:
import pandas as pd
import re
import glob
from sqlalchemy import create_engine
from datetime import datetime
import pytz

In [90]:
# folder_path = 'I:/Expense tracking/Apple Pay'
# folder_path='D:/Expense tracking/Apple Pay'
folder_path='Apple Pay'
file_pattern = folder_path + '/*.csv'  # Adjust the pattern if using other formats, e.g., '*.xlsx'

# Use glob to find all files matching the pattern
file_list = glob.glob(file_pattern)

# Read all files and concatenate them into a single DataFrame
df_apple_pay = pd.concat((pd.read_csv(file) for file in file_list), ignore_index=True)

In [91]:
df_apple_pay

,Transaction Date,Clearing Date,Description,Merchant,Category,Type,Amount (USD),Purchased By
0,04/29/2024,04/30/2024,GOODYEAR ASC #4181 8012 COIT ROAD PLANO 75024 ...,Goodyear,Other,Purchase,117.49,Sandeep Rani Pradeep Kumar
1,04/30/2024,04/30/2024,ACH DEPOSIT INTERNET TRANSFER FROM ACCOUNT END...,Ach Deposit Internet Transfer From Account End...,Payment,Payment,-165.62,Sandeep Rani Pradeep Kumar
2,04/23/2024,04/24/2024,DUNKIN #346310 7317 PARKWOOD BOULEVARD PLANO 7...,Dunkin',Restaurants,Purchase,2.81,Sandeep Rani Pradeep Kumar
3,04/22/2024,04/23/2024,FEDEX OFFIC798000079898290 STATE HWY 121 FRISC...,FedEx,Other,Purchase,6.54,Sandeep Rani Pradeep Kumar
4,04/19/2024,04/20/2024,WDW TICKETS 1375 E BUENA VISTA DR LAKE BUENA V...,WDW Tickets,Entertainment,Purchase,381.28,Sandeep Rani Pradeep Kumar
...,...,...,...,...,...,...,...,...
63,11/05/2023,11/05/2023,HELLO! INDIA 8740 OHIO DR PLANO 75024 TX USA,Hello! India,Restaurants,Purchase,13.90,Sandeep Rani Pradeep Kumar
64,10/27/2023,10/28/2023,DUNKIN #346310 7317 PARKWOOD BOULEVARD PLANO 7...,Dunkin',Restaurants,Purchase,2.69,Sandeep Rani Pradeep Kumar
65,10/22/2023,10/24/2023,STARBUCKS STORE 6411 5760 LEGACY DR STE B6 PLA...,Starbucks,Restaurants,Purchase,2.87,Sandeep Rani Pradeep Kumar
66,10/23/2023,10/23/2023,KRISPY KREME #385 8815 PRESTON ROAD FRISCO 750...,Krispy Kreme,Restaurants,Purchase,5.79,Sandeep Rani Pradeep Kumar


In [92]:
# df_apple_pay['Transaction Date'] = pd.to_datetime(df_apple_pay['Transaction Date'])
# df_apple_pay['Transaction Date']=df_apple_pay['Transaction Date'].dt.strftime('%m/%d/%y')
# # df_apple_pay['Transaction Date']=pd.to_datetime(df_apple_pay['Transaction Date'], format='%m/%d/%y')

In [93]:
df_apple_pay=df_apple_pay[['Transaction Date','Description','Category','Amount (USD)','Merchant']]
df_apple_pay['Institution']='Apple Pay Credit Card'
df_apple_pay['Is Hidden']='No'
df_apple_pay['Is Pending']='No'
df_apple_pay['Account']='APPLE PAY'


C:\Users\sande\AppData\Local\Temp\ipykernel_47696\3459581008.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_apple_pay['Institution']='Apple Pay Credit Card'
C:\Users\sande\AppData\Local\Temp\ipykernel_47696\3459581008.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_apple_pay['Is Hidden']='No'


In [94]:
df_apple_pay.rename(columns={"Merchant":"Standardized Description","Amount (USD)":"Amount"},inplace=True)
dtype_dict = {
    'Transaction Date':'datetime64[ns]',
    'Description': 'string',
    'Institution': 'string',
    'Account': 'string',
    'Category': 'string',
    'Is Hidden': 'string',
    'Is Pending': 'string',
    'Amount': 'float64',
    'Standardized Description': 'string'
}
df_apple_pay = df_apple_pay.astype(dtype_dict)
# df_apple_pay['Transaction Date']=df_apple_pay['Transaction Date'].dt.strftime('%m/%d/%y')
df_apple_pay=df_apple_pay[['Transaction Date','Description','Institution','Account','Category','Is Hidden','Is Pending','Amount','Standardized Description']]


In [95]:
df_apple_pay.head()

,Transaction Date,Description,Institution,Account,Category,Is Hidden,Is Pending,Amount,Standardized Description
0,2024-04-29,GOODYEAR ASC #4181 8012 COIT ROAD PLANO 75024 ...,Apple Pay Credit Card,APPLE PAY,Other,No,No,117.49,Goodyear
1,2024-04-30,ACH DEPOSIT INTERNET TRANSFER FROM ACCOUNT END...,Apple Pay Credit Card,APPLE PAY,Payment,No,No,-165.62,Ach Deposit Internet Transfer From Account End...
2,2024-04-23,DUNKIN #346310 7317 PARKWOOD BOULEVARD PLANO 7...,Apple Pay Credit Card,APPLE PAY,Restaurants,No,No,2.81,Dunkin'
3,2024-04-22,FEDEX OFFIC798000079898290 STATE HWY 121 FRISC...,Apple Pay Credit Card,APPLE PAY,Other,No,No,6.54,FedEx
4,2024-04-19,WDW TICKETS 1375 E BUENA VISTA DR LAKE BUENA V...,Apple Pay Credit Card,APPLE PAY,Entertainment,No,No,381.28,WDW Tickets


In [96]:
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)

In [97]:
def convert_accounting_to_decimal(amount):
    # Remove parentheses and commas
    amount = amount.replace('$', '').replace('(', '-').replace(')', '').replace(',', '')
    # Convert to float
    return float(amount)

In [98]:
folder_path='All Transactions'
file_pattern = folder_path + '/*.csv'  # Adjust the pattern if using other formats, e.g., '*.xlsx'

# Use glob to find all files matching the pattern
file_list = glob.glob(file_pattern)

# Read all files and concatenate them into a single DataFrame
df = pd.concat((pd.read_csv(file) for file in file_list), ignore_index=True)


# file_names = ['Transactions 2022_06-2023_06.csv', 'Transactions 2023_06-2024_06.csv']
# folder_path = 'D:/Expense tracking'
# # folder_path='D:/Personal Projects/Expense tracking'
# file_pattern = folder_path + '/*.csv'  # Adjust the pattern if using other formats, e.g., '*.xlsx'

# # Use glob to find all files matching the pattern
# file_list = glob.glob(file_pattern)
# print(file_list)

# # Load and concatenate the files into a single DataFrame
# df = pd.concat([pd.read_csv(file) for file in file_names], ignore_index=True)


In [99]:
df['Amount']=df['Amount'].apply(convert_accounting_to_decimal)

In [100]:
df['Standardized Description']=''
df.rename(columns={"Date":"Transaction Date"},inplace=True)


In [101]:

dtype_dict = {
    'Transaction Date':'datetime64[ns]',
    'Description': 'string',
    'Institution': 'string',
    'Account': 'string',
    'Category': 'string',
    'Is Hidden': 'string',
    'Is Pending': 'string',
    'Amount': 'float64',
    'Standardized Description': 'string'
}
df = df.astype(dtype_dict)
# len(df)

In [102]:
df['Account'].unique()

<StringArray>
[          'CHASE FREEDOM',           'GOLDMAN SACHS',
 'CHARLES SCHWAB CHECKING',          'CHASE CHECKING',
               'ROBINHOOD',           'CHASE SAVINGS',
                'DISCOVER',                    'AMEX',
    'WELLS FARGO CHECKING']
Length: 9, dtype: string

In [103]:
len(df)

2703

In [104]:
df_full=pd.concat([df,df_apple_pay],ignore_index=True)

In [105]:
len(df_full)

2771

In [106]:
# pd.reset_option('display.max_rows')
# pd.reset_option('display.max_columns')
df_full[df_full['Account']=='APPLE PAY']['Category'].unique()
df_full[df_full['Category']=='Medical']

,Transaction Date,Description,Institution,Account,Category,Is Hidden,Is Pending,Amount,Standardized Description
1159,2024-08-17,CURTIS CULLWELL CENTER,Chase - via Direct Web API,CHASE FREEDOM,Medical,No,No,-324.00,
1195,2024-08-05,UROLOGY CLINICS OF N,Chase - via Direct Web API,CHASE FREEDOM,Medical,No,No,-128.98,
1208,2024-08-01,MED TEXASHEALTH RESOUR,Chase - via Direct Web API,CHASE FREEDOM,Medical,No,No,-803.36,
1336,2024-06-28,KAZMO BRAIN CENTER LFRISCO TX,American Express - Amex Account - via Direct W...,AMEX,Medical,No,No,-299.00,
2560,2023-08-08,NAYAX COPY SERVICES,Chase - via Direct Web API,CHASE FREEDOM,Medical,No,No,-3.00,
2561,2023-08-08,NAYAX COPY SERVICES,Chase - via Direct Web API,CHASE FREEDOM,Medical,No,No,-0.45,
2721,2024-02-22,THE JOINT PRESTON ROA8240 PRESTON ROAD STE 165...,Apple Pay Credit Card,APPLE PAY,Medical,No,No,118.00,The Joint
2738,2024-06-11,THE JOINT PRESTON ROA8240 PRESTON ROAD STE 165...,Apple Pay Credit Card,APPLE PAY,Medical,No,No,89.00,The Joint
2747,2024-03-22,THE JOINT PRESTON ROA8240 PRESTON ROAD STE 165...,Apple Pay Credit Card,APPLE PAY,Medical,No,No,89.00,The Joint


In [107]:
df_full.head()

,Transaction Date,Description,Institution,Account,Category,Is Hidden,Is Pending,Amount,Standardized Description
0,2023-06-30,DUNKIN,Chase - via Direct Web API,CHASE FREEDOM,Coffee,No,No,-5.67,
1,2023-06-30,INTEREST PAID,Goldman Sachs - Marcus - via Direct Web Access,GOLDMAN SACHS,Interest Income,No,No,77.55,
2,2023-06-30,INTEREST PAID,Charles Schwab US,CHARLES SCHWAB CHECKING,Interest Income,No,No,0.02,
3,2023-06-30,TELADOC HEALTH I DIRECT DEP PPD ID,Chase - via Direct Web API,CHASE CHECKING,Income,No,No,3743.32,
4,2023-06-30,ROBINHOOD DEBITS WEB ID,Chase - via Direct Web API,CHASE CHECKING,Investment Savings,No,No,-300.00,


In [108]:
df_full['Category'].unique()

<StringArray>
[                      'Coffee',              'Interest Income',
                       'Income',           'Investment Savings',
         'Investment Transfers',                    'Transfers',
          'Credit Card Payment',           'Restaurants/Dining',
               'Online Payment',                     'Shopping',
                    'Groceries',      'Fast Food & Convenience',
       'Energy, Gas & Electric',                   'Gas & Fuel',
                     'Bank Fee',                     'Cash/ATM',
                   'Air Travel',                          'Gym',
                'Entertainment',                 'Hair & Nails',
                      'Savings',      'Phone, Internet & Cable',
                   'Rental Car',                        'Tolls',
                    'Rideshare',                'Subscriptions',
                 'Auto Service',                         'Rent',
          'Shipping & Handling',               'Fees & Charges',
           

In [109]:
patterns_map = {
    'COFFEE': [r'STARBUCKS', r'DUNKIN',r'PANERA'],
    'SHOPPING':[r'WALMART',r'TARGET'],
    'RENT':[r'BELL WEB',r'SUMMIT',r'PALMS ASSOCIATES',r'BILTRP'],
    'ENERGY, GAS & ELECTRIC':[r'TXU'],
    'RESTAURANTS/DINING':[r'HELLO INDIA',r'HELLO! INDIA'],
    'SUBSCRIPTIONS':[r'ENVATO',r'STRATA',r'VIDIQ',r'YOUTUBE',r'HUNTER.IO'],
    'PAYMENT':[r'USCIS',r'ZELLE PAYMENT TO'],
    'INCOME':[r'ZELLE PAYMENT FROM'],
    'ENTERTAINMENT':[r'SIXFLAGS'],
    'GYM':[r'CRUNCH']
}

# Function to map the new category
def map_category(row):
    description_upper = row['Description'].upper()  # Convert to uppercase for case-insensitive matching
    for category, patterns in patterns_map.items():
        if any(re.search(pattern, description_upper) for pattern in patterns):
            return category
    return row['Category'].upper()
    
df_full['Category'] = df_full.apply(map_category, axis=1)



In [110]:
master_category_dict={
'SHOPPING/GROCERIES':['Groceries','Grocery','Shopping','Furniture & Home Decor','Merchandise/Misc','Gifts','Clothing','Home Improvement/Maintenance','Books','Electronics & Software'],
'ENTERTAINMENT':['Entertainment','Air Travel','Hotel','Rental Car','Travel & Vacation'],
'FOOD AND DINING':['Restaurants','Restaurants/Dining','Fast Food & Convenience',	'Coffee','Food','Alcohol & Bars'],
'SUBSCRIPTIONS':['Subscriptions','Education'],
'HEALTH':['Gym'],
'BILLS AND UTILITIES':['Phone, Internet & Cable','Gas & Fuel','Gas','Insurance','Auto Service','Energy, Gas & Electric','Auto & Transport',
                       	'Water'	,'Bills & Utilities','Rent','Tolls','Rideshare','Parking','Pharmacy','Hair & Nails','Medical','Household Services'],
'MISCELLANEOUS':['Transfers','Credit Card Payment','Savings','Investment Savings','Investment Transfers','Online Payment','Bank Fee'
                 	,'Fees & Charges','Shipping & Handling','Taxes','Cash/ATM','Charity','Payment','Other'],
'INCOME':['Interest Income','Paycheck/Salary','Tax Refund','Income']
    
}

def map_category(row):
    for category, patterns in master_category_dict.items():
        if any(pattern.lower() in row['Category'].lower() for pattern in patterns):
            return category
    return 'OTHER'
    
df_full['Master Category'] = df_full.apply(map_category, axis=1)


In [111]:
df_full['Master Category'].unique()

array(['FOOD AND DINING', 'INCOME', 'MISCELLANEOUS', 'SHOPPING/GROCERIES',
       'BILLS AND UTILITIES', 'ENTERTAINMENT', 'HEALTH', 'SUBSCRIPTIONS',
       'OTHER'], dtype=object)

In [112]:
df_full.head()

,Transaction Date,Description,Institution,Account,Category,Is Hidden,Is Pending,Amount,Standardized Description,Master Category
0,2023-06-30,DUNKIN,Chase - via Direct Web API,CHASE FREEDOM,COFFEE,No,No,-5.67,,FOOD AND DINING
1,2023-06-30,INTEREST PAID,Goldman Sachs - Marcus - via Direct Web Access,GOLDMAN SACHS,INTEREST INCOME,No,No,77.55,,INCOME
2,2023-06-30,INTEREST PAID,Charles Schwab US,CHARLES SCHWAB CHECKING,INTEREST INCOME,No,No,0.02,,INCOME
3,2023-06-30,TELADOC HEALTH I DIRECT DEP PPD ID,Chase - via Direct Web API,CHASE CHECKING,INCOME,No,No,3743.32,,INCOME
4,2023-06-30,ROBINHOOD DEBITS WEB ID,Chase - via Direct Web API,CHASE CHECKING,INVESTMENT SAVINGS,No,No,-300.00,,MISCELLANEOUS


In [113]:
desc_map = {
    'STARBUCKS': [r'STARBUCKS'],
    'DUNKIN': [r'DUNKIN'],
    '7-ELEVEN':[r'7-ELEVEN'],
    'WALMART':[r'WALMART',r'WM',r'WAL-MART'],
    'TARGET':[r'TARGET'],
    'HOME DEPOT':[r'HOME DEPOT'],
    'AMAZON':[r'AMAZON',r'AMZN'],
    'COSTCO':[r'COSTCO'],
    'HELLO INDIA': [r'HELLO INDIA',r'HELLO! INDIA'],
    'PANERA':[r'PANERA'],
    'TELADOC':[r'TELADOC'],
    'SIGNIFY':[r'SIGNIFY'],
    'CMA CGM':[r'CMA CGM']
}

def map_description(row):
    description_upper = row['Description'].upper()  # Convert to uppercase for case-insensitive matching
    for category, patterns in desc_map.items():
        if any(re.search(pattern, description_upper) for pattern in patterns):
            return category
    return row['Description'].upper()
    

# Apply the function to create a new column 'Mapped Description'
df_full['Standardized Description'] = df_full.apply(map_description,axis=1)


Filter out 7 ELEVEN as it needs cleaning

In [114]:

filtered_df = df_full[~(df_full['Description'].str.match('7-ELEVEN', case=False, na=False) & (df_full['Category'] == 'FAST FOOD & CONVENIENCE'))]


7-ELEVEN 

In [115]:
df_7_eleven = df_full[df_full['Description'].str.match('7-ELEVEN', case=False, na=False) & (df_full['Category']=='FAST FOOD & CONVENIENCE')]
df_7_eleven.loc[:,'Amount']=df_7_eleven['Amount']+2.0
df_7_eleven_coffee=df_7_eleven.copy()
df_7_eleven_coffee.loc[:,'Amount']=-2.00
df_7_eleven_coffee.loc[:,'Category']='COFFEE'
df_7_eleven_coffee.loc[:,'Master Category']='FOOD AND DINING'

df_7_eleven_transformed = pd.concat([df_7_eleven_coffee, df_7_eleven], ignore_index=True)


In [116]:
df_final=pd.concat([filtered_df, df_7_eleven_transformed], ignore_index=True)

In [117]:
df_final.loc[:,"insert_date"]=datetime.strptime(datetime.now(pytz.timezone('US/Central')).strftime('%Y-%m-%d %H:%M:%S'),'%Y-%m-%d %H:%M:%S')

In [118]:
len(df_final)
df_final.head(100)
len(df_final[(df_final['Transaction Date'] >= '2022-06-01') & (df_final['Transaction Date'] <= '2024-07-01')])
sum(df_final.isnull().any(axis=1))


0

In [119]:
len(df_final)

2833

In [120]:
server = 'localhost\\sqlexpress'
database = 'database'

engine = create_engine(f'mssql+pyodbc://@{server}/{database}?driver=ODBC+Driver+17+for+SQL+Server')

In [121]:
df_final.to_sql('expense_tracking_data', con=engine, if_exists='replace', index=False)
print("Data Refreshed in Table")

Data Refreshed in Table


In [122]:
# filtered_df.groupby('Standardized Description')['Amount'].sum().sort_values(ascending=True)
# filtered_df['Amount'].sum()
# df_final.to_csv('Transactions Full.csv')

